# `MIDAS`

$ \textrm{We want to maximize the media-driven NOS of a media plan, by deciding the amount of budget per vehicle that we should spend} $
$ \textrm{for a given brand/market combination on a given period. Specifically,}$<br>
<br>
$ \textrm{Given } \textit{N} \textrm{ response curves } \textit{i}:$<br><br>
&emsp;&emsp;$ \textrm{max} \sum \limits _{i=1} ^{N} f^{NOS}_{i}(\tilde{x}_{i})$<br>
&emsp;&emsp;&emsp;$ s.t. \; 1 \geq \tilde{x}_{i} \geq 0 \;\;\;\;\;\;\;\;\;\;\;\;  (1)$ <br>
&emsp;&emsp;&emsp;&emsp;&emsp;$    \; \sum _{i=1} ^{N} y_{i} \leq budget \;\;\; (2)$<br><br>

&emsp;&emsp;$ \textrm{where:} $<br> 
&emsp;&emsp;&emsp;&emsp;$ f^{NOS}_{i} = \textrm{ fitted } \textit{NOS}  \textrm{ function for the } \textit{i} \textrm{-th media vehicle}$<br>
&emsp;&emsp;&emsp;&emsp;$ \tilde{x}_{i} = \textrm{ }\mathbf{scaled} \textrm{ total impressions} \textrm{: } \tilde{x}_{i} =  \frac{x_{i} - x_{min}}{x_{max}-x_{min}} $<br>
&emsp;&emsp;&emsp;&emsp;$ x_{i} = \textrm{ total } \mathbf{impressions} \textrm{ given spend } y_{i} \textrm{: } x_{i} = g^{impressions}_{i}(y_{i}) $<br>
&emsp;&emsp;&emsp;&emsp;$ g^{impressions}_{i} = \textrm{ fitted } \textit{impressions}  \textrm{ function for the } \textit{i} \textrm{-th media vehicle}$<br>
&emsp;&emsp;&emsp;&emsp;$ y_{i} = \textrm{ total } \mathbf{spend} \textrm{ on the } \textit{i} \textrm{-th media vehicle}$<br>


In [1]:
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import curve_fit
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy
import math

In [2]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
from funcs.proi import *

ModuleNotFoundError: No module named 'funcs'

---
## Load data

In [ ]:
## config
config = pd.read_excel('midas-pe-pantene-v2.xlsx')
market = config.market.unique()[0]
brand = config.brand.unique()[0]

In [ ]:
# data
curves = pd.read_csv('input_notebook\database-curves.csv')
financials =  pd.read_excel('input_notebook\database-financials-v2-pe.xlsx')

In [ ]:
# set entity
curves = curves.query(f'market == "{market}" & brand == "{brand}"')
vehicles = set([v for v in curves.vehicle.unique()])

data_dict = {v: {'data': curves.query(f'vehicle == "{v}"')[['Weekly Support','Prediction']]} for v in vehicles}

for k in data_dict.keys():
    data_dict[k]['cpp'] = financials[(financials.vehicle==k)&(financials.market==market)&(financials.brand==brand)&(financials.metric=='cpp')]['value'].values[0]
    
    try:
        data_dict[k]['spend'] = config[(config.vehicle==k)&(config.metric=='spend')]['value'].values[0]
        data_dict[k]['weeks'] = config[(config.vehicle==k)&(config.metric=='weeks')]['value'].values[0]
        data_dict[k]['ix_spend'] = config[(config.vehicle==k)&(config.metric=='ix_spend')]['value'].values[0]
        data_dict[k]['ix_nos'] = config[(config.vehicle==k)&(config.metric=='ix_nos')]['value'].values[0]
        data_dict[k]['imp'] = data_dict[k]['spend'] / data_dict[k]['cpp'] * data_dict[k]['ix_spend'] / data_dict[k]['weeks']
    except:
        print(f'Missing vehicle: {k}')
        data_dict[k]['spend'] = 0
        data_dict[k]['weeks'] = 52
        data_dict[k]['ix_spend'] = financials[(financials.vehicle==k)&(financials.market=='all')&(financials.metric=='ix_spend')]['value'].values[0]
        data_dict[k]['ix_nos'] = financials[(financials.vehicle==k)&(financials.market=='all')&(financials.metric=='ix_nos')]['value'].values[0]
        data_dict[k]['imp'] = 0

NOS_PER_SU = financials[(financials.vehicle=='total')&(financials.market==market)&(financials.brand==brand)&(financials.metric=='nos_per_su')]['value'].values[0]
TOTAL_SPEND = np.sum([data_dict[k]['spend'] for k in data_dict.keys()])

---
## Fit curves

In [ ]:
# Plot curves
plot_curves(data_dict)

In [ ]:
funcs = [hill, poly2, log_, cdf_, weibull]
data_dict = fit_curves(funcs=funcs, data_dict=data_dict, poly1d=True)

In [ ]:
plot_fitted_curves(data_dict)

---
## `MIDAS NOS·Translator`

In [ ]:
def objective(x:list) -> float:
    '''
    Total NOS per SU given impressions per vehicle
    
    Args:
        x: a list with impressions per vehicle

    Returns:
        Total NOS per SU achieved
    '''
    return -(NOS_PER_SU * \
             np.sum([get_nos(x[i], data_dict, k) for i, k in enumerate(data_dict.keys())])
            )

In [ ]:
actuals = np.array([data_dict[k]['scaler'].transform([[data_dict[k]['imp']]])[0][0] \
              for k in data_dict.keys()])
print(f'Current NOS: ${-objective(actuals):,.2f}')
print(f'Current ROI: {-objective(actuals)/TOTAL_SPEND:,.2f}')
print(f'Total SU: {-objective(actuals)/NOS_PER_SU:,.2f}')

print()
print('spent')
print('-'*20)
for i, k in enumerate(data_dict.keys()):
    print(f'{k}: ${round(get_spend(actuals[i], data_dict, k)):,.2f}')

In [ ]:
plot_actual_curves(data_dict=data_dict, x=actuals)

In [ ]:
keys = [k for k in data_dict.keys()]
values = [round(get_spend(actuals[i], data_dict, k)) for i, k in enumerate(data_dict.keys())]

plt.pie(values, labels=keys, colors=sns.color_palette('bright'), autopct='%.0f%%')
plt.title('Budget split')
plt.show()

---

## `MIDAS NOS·Optimizer`

In [ ]:
def constraint1(x:list) -> float:
    '''
    Total spend given impressions per vehicle
    
    Args:
        x: a list with impressions per vehicle

    Returns:
        Exceeding budget spent
    '''
    return TOTAL_SPEND - \
            np.sum([get_spend(x[i], data_dict, k) for i, k in enumerate(data_dict.keys())])

In [ ]:
initial_guess = [np.random.random() for _ in range(len(data_dict.keys()))]
cons = [{'type': 'ineq', 'fun': constraint1}]

result = minimize(objective, 
                  initial_guess, 
                  method = 'trust-constr',
                  constraints=cons,
                  bounds=[(0.01, 1.1) for _ in range(len(data_dict.keys()))],
                  options={'maxiter': 2_000, 'verbose':2},
                 )

print(result.message)
print()
print(result.x)

In [ ]:
EPS = 1

# re-check constraints!
spend_const = (np.sum([get_spend(result.x[i], data_dict, k) for i, k in enumerate(data_dict.keys())]) <= TOTAL_SPEND + EPS)
bound_const = (all([(i >= 0) & (i <= 1) for i in [*result.x]]))
objective_const = (-objective(actuals) < -objective(result.x))

print('constraints')
print('-'*13)
print(f'Budget: \t{spend_const}')
print(f'Bounds: \t{bound_const}')
print(f'Objective: \t{objective_const}')

In [ ]:
print(f'Result (NOS): ${-objective(result.x):,.2f} (+${round(-objective(result.x)+objective(actuals)):,.0f})')
print(f'Result (ROI): {-objective(result.x)/TOTAL_SPEND:,.2f} ({(-objective(result.x)/TOTAL_SPEND)-(-objective(actuals)/TOTAL_SPEND):,.2f})')
print()

print('NOS:')
print('-'*20)
for i, k in enumerate(data_dict.keys()):
    print(f'{k}: ${round(get_nos(result.x[i], data_dict, k) * NOS_PER_SU):,.2f}')

print()

print('spent')
print('-'*20)
for i, k in enumerate(data_dict.keys()):
    print(f'{k}: ${round(get_spend(result.x[i], data_dict, k)):,.2f}')

In [ ]:
keys = [k for k in data_dict.keys()]
values = [round(get_spend(result.x[i], data_dict, k)) for i, k in enumerate(data_dict.keys())]

plt.pie(values, labels=keys, colors=sns.color_palette('bright'), autopct='%.0f%%')
plt.title('Budget split')
plt.show()

In [ ]:
plot_solution_curves(data_dict=data_dict, x=result.x, actual=actuals)

---